In [1]:
# https://github.com/cda-tum/qcec
!pip install mqt.qcec==2.0.0rc1
!pip install qiskit
#!pip install mqt.ddsim 

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit.test.mock import FakeAthens
from mqt import qcec

if __name__ == "__main__":
    # create your quantum circuit
    circ = QuantumCircuit(3)
    circ.h(0)
    circ.cx(0, 1)
    circ.cx(0, 2)
    circ.measure_all()
    print(circ.draw(fold=-1))

    # compile circuit to 5 qubit London Architecture
    circ_comp = transpile(circ, backend=FakeAthens())
    print(circ_comp.draw(fold=-1))

    # initialize the equivalence checker
    ecm = qcec.EquivalenceCheckingManager(circ, circ_comp, )

    # set the application scheme to be based off a profile
    ecm.set_application_scheme('gate_cost')
    ecm.set_gate_cost_profile('dummy.profile')

    # execute the check
    ecm.run()

    # obtain the result
    print(ecm.equivalence())

    # obtain runtime
    results = ecm.get_results()
    print('Took:', results.check_time, '[s]')


        ┌───┐           ░ ┌─┐      
   q_0: ┤ H ├──■────■───░─┤M├──────
        └───┘┌─┴─┐  │   ░ └╥┘┌─┐   
   q_1: ─────┤ X ├──┼───░──╫─┤M├───
             └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ──────────┤ X ├─░──╫──╫─┤M├
                  └───┘ ░  ║  ║ └╥┘
meas: 3/═══════════════════╩══╩══╩═
                           0  1  2 
global phase: π/4
                                                     ┌───┐           ░    ┌─┐   
      q_2 -> 0 ───────────────────────────────────■──┤ X ├──■────────░────┤M├───
                                           ┌───┐┌─┴─┐└─┬─┘┌─┴─┐┌───┐ ░    └╥┘┌─┐
      q_1 -> 1 ────────────────────────────┤ X ├┤ X ├──■──┤ X ├┤ X ├─░─────╫─┤M├
               ┌─────────┐┌────┐┌─────────┐└─┬─┘└───┘     └───┘└─┬─┘ ░ ┌─┐ ║ └╥┘
      q_0 -> 2 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■───────────────────■───░─┤M├─╫──╫─
               └─────────┘└────┘└─────────┘                          ░ └╥┘ ║  ║ 
ancilla_0 -> 3 ──────────────────────────────────────────────────────░──╫──╫──╫─
          

In [3]:
# NO PARAMETER, ONE QUBIT GATES
nativelySupportedGates = {"i","x", "y", "z", "h", "s", "sdg", "t", "tdg", "sx", "sxdg"}
circuit = QuantumCircuit(1)
for element in nativelySupportedGates:
    circuit = QuantumCircuit(1)
    a = (element, 0)
    getattr(circuit, a[0])(a[1])
    #print(circuit.size())
    
    circ_comp = transpile(circuit, backend=FakeAthens())
    #print(circ_comp.size())
    print(element, 0, circ_comp.size(), sep =' ')
#circuit.draw()

h 0 3
t 0 1
z 0 1
id 0 0
sx 0 1
sdg 0 1
s 0 1
i 0 0
y 0 2
tdg 0 1
sxdg 0 3
x 0 1


In [4]:
# NO PARAMETER, TWO QUBIT GATES
nativelySupportedGates = {"swap", "iswap"}
circuit = QuantumCircuit(2)
for element in nativelySupportedGates:
    circuit = QuantumCircuit(2)
    a = (element, 0, 1)
    getattr(circuit, a[0])(a[1], a[2])
    #print(circuit.size())
    
    circ_comp = transpile(circuit, backend=FakeAthens())
    #print(circ_comp.size())
    print(element, 0, circ_comp.size(), sep =' ')
#circuit.draw()

iswap 0 9
swap 0 3


In [5]:
# GATE WITH CONTROL BITS
nativelySupportedGates = {"cx"}
circuit = QuantumCircuit(1)
circuit.x(0)
args_arr = [0]
for i in range(15):
    custom = circuit.to_gate().control(i+1)
    new_circuit = QuantumCircuit(2+i)
    args_arr.append(i+1)
    new_circuit.append(custom, args_arr)
    #print(new_circuit.size())
    #print(new_circuit)
    
    circ_comp = transpile(new_circuit, basis_gates=["id", "rz", "sx", "x", "cx"])
    #print(circ_comp.size())
    print('x', i+1, circ_comp.size(), sep =' ')
#circuit.draw()

x 1 1
x 2 18
x 3 34
x 4 97
x 5 190
x 6 382
x 7 766
x 8 1534
x 9 3070
x 10 6142
x 11 12286
x 12 24574
x 13 49150
x 14 98302
x 15 196606


In [6]:
from qiskit import QuantumCircuit
circuit = QuantumCircuit(2)
circuit.draw()
a = ('cx', 0, 1)
getattr(circuit, a[0])(a[1], a[2])
circuit.draw()

q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘

In [12]:
from qiskit import QuantumCircuit
circuit = QuantumCircuit(2)
circuit.draw()
a = ('id', 0)
getattr(circuit, a[0])(a[1])
circuit.draw()

┌───┐
q_0: ┤ I ├
     └───┘
q_1: ─────